In [1]:
import time 
import json 
import random 
from datetime import datetime
from data_generator import generate_message
from kafka import KafkaProducer
import alpaca_trade_api as tradeapi
from alpaca_trade_api.stream import Stream
from alpaca_trade_api.common import URL
import threading
import websocket

# Messages will be serialized as JSON 
def serializer(message):
    return json.dumps(message).encode('utf-8')


# Kafka Producer
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=serializer
)

data_url = 'wss://data.alpaca.markets'
base_url = 'https://paper-api.alpaca.markets'
api_key= 'PKZEXWB0J5CEU2JJZXV8'
api_secret = 'kPy7mGsSnzeFnTgpMKFAQG58blbwqYgPiTeEeUPo'

def alpaca():
    def on_open(ws):
        print("opened")
        auth_data = {"action": "auth", "key": api_key, "secret": api_secret}
        print(auth_data)
        ws.send(json.dumps(auth_data))
        listen_message = {"action": "subscribe", "bars": ["AAPL"]}       
        ws.send(json.dumps(listen_message))


    def on_message(ws, message):        
        allowed=['b']
        #print(f'Sending @ {datetime.now()} | Message = {str(message)[1:-1]}')
        msgjson = json.loads(message[1:-1])
        if(msgjson['T'] in allowed):  
            print(msgjson)
            body = {
                "Close": str(msgjson["c"]),        
                "Open": str(msgjson["o"]),
                "High": str(msgjson["h"]),
                "Low": str(msgjson["l"]),
                "Volume": str(msgjson["v"]),
                "Trade_count": str(msgjson["n"]),
                "vwap": str(msgjson["vw"])
            }            
            producer.send('messages', body)
        
        # Sleep for a random number of seconds
        time_to_sleep = random.randint(1, 11)
        time.sleep(time_to_sleep)

    def on_close(ws, close_status, closemessage):
        print("closed connection ", closemessage)        
        print ("Retry : %s" % time.ctime())
        time.sleep(1)
        connect_websocket() # retry per 10 seconds

    def on_error(ws, message):
        print("ERROR connection ", message)

    def connect_websocket():
        socket = "wss://stream.data.alpaca.markets/v2/iex"
        ws = websocket.WebSocketApp(socket, on_open=on_open, on_message=on_message, on_close=on_close, on_error=on_error)
        ws.run_forever()

    connect_websocket()

def aftermarket():
    while(1):
        with open('dummy') as f:
            lines = f.readlines()
            for line in lines:              
                print(json.loads(line))
                producer.send('messages', json.loads(line))
                # Sleep for a random number of seconds
                time_to_sleep = random.randint(1, 10)
                time.sleep(time_to_sleep)


if __name__ == '__main__':

    api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')
    clock = api.get_clock()
    if clock.is_open:
        alpaca()
    else:
        aftermarket()

    

opened
{'action': 'auth', 'key': 'PKZEXWB0J5CEU2JJZXV8', 'secret': 'kPy7mGsSnzeFnTgpMKFAQG58blbwqYgPiTeEeUPo'}
{'T': 'b', 'S': 'AAPL', 'o': 158.15, 'c': 158.085, 'h': 158.23, 'l': 158.07, 'v': 9718, 't': '2021-11-26T15:17:00Z', 'n': 88, 'vw': 158.136535}
{'T': 'b', 'S': 'AAPL', 'o': 158.07, 'c': 157.94, 'h': 158.07, 'l': 157.82, 'v': 8369, 't': '2021-11-26T15:18:00Z', 'n': 84, 'vw': 157.939802}
{'T': 'b', 'S': 'AAPL', 'o': 157.96, 'c': 158.09, 'h': 158.12, 'l': 157.96, 'v': 5913, 't': '2021-11-26T15:19:00Z', 'n': 60, 'vw': 158.022128}
{'T': 'b', 'S': 'AAPL', 'o': 158.08, 'c': 158.04, 'h': 158.15, 'l': 158.04, 'v': 7834, 't': '2021-11-26T15:20:00Z', 'n': 62, 'vw': 158.090394}
{'T': 'b', 'S': 'AAPL', 'o': 158.03, 'c': 158.095, 'h': 158.095, 'l': 157.97, 'v': 8919, 't': '2021-11-26T15:21:00Z', 'n': 69, 'vw': 158.021165}
{'T': 'b', 'S': 'AAPL', 'o': 158.005, 'c': 157.955, 'h': 158.02, 'l': 157.91, 'v': 2988, 't': '2021-11-26T15:22:00Z', 'n': 33, 'vw': 157.9496}
{'T': 'b', 'S': 'AAPL', 'o':

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\shanm\anaconda3\lib\site-packages\websocket\_app.py", line 369, in run_forever
    dispatcher.read(self.sock.sock, read, check)
  File "C:\Users\shanm\anaconda3\lib\site-packages\websocket\_app.py", line 68, in read
    r = self.select()
  File "C:\Users\shanm\anaconda3\lib\site-packages\websocket\_app.py", line 82, in select
    r = sel.select(self.ping_timeout)
  File "C:\Users\shanm\anaconda3\lib\selectors.py", line 323, in select
    r, w, _ = self._select(self._readers, self._writers, [], timeout)
  File "C:\Users\shanm\anaconda3\lib\selectors.py", line 314, in _select
    r, w, x = select.select(r, w, w, timeout)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\shanm\anaconda3\lib\site-packages\websocket\_app.py", line 369, in run_forever
    dispatcher.read(self.sock.sock, read, check)
  File "C:\Users\shanm\anaconda3\lib\site-packages\we

TypeError: object of type 'NoneType' has no len()